## Drafts for visual analysis of benchmarking results from identification workflow

In [1]:
import glob
from pyteomics import mzid
import numpy as np
import matplotlib.pyplot as plt
import os
from venn import venn

In [11]:
# read in 4 mzid files (one for each merging method)
files = glob.glob('../data/*.mzid')
# names of methods
basenames = [os.path.basename(name) for name in files]

In [ ]:
dfs = [mzid.DataFrame(f) for f in files]

In [ ]:
def evalue_hist(df, title):
    np.log10(df['MS-GF:EValue']).hist(bins=100)
    plt.title(title)

In [ ]:
# simple histograms of PSM evalues
plt.figure(figsize=(16, 10))
for i, (name, df) in enumerate(zip(basenames, dfs), 1):
    plt.subplot(2, 2, i)
    evalue_hist(df, name)
    plt.xlabel('log(e-value)')
plt.tight_layout()

### subtract the mean from 4 histograms and plot the difference ###

In [ ]:
Nbins = 50
bins = np.linspace(-14, 0, Nbins)
counts = np.empty((4, Nbins-1))
for i, df in enumerate(dfs):
    counts[i, :] = np.histogram(np.log10(df['MS-GF:EValue']), bins=bins)[0]

In [ ]:
mean = counts.mean(axis=0)
plt.figure(figsize=(18, 9))
plt.step(bins[:-1], (counts-mean).T)
plt.xlabel('log(e-value)')
plt.legend(basenames)

### Bar plots of number of PSMs and peptides

In [ ]:
npsms = [df.shape[0] for df in dfs]
plt.bar(range(4), npsms)
plt.xticks([])
plt.title('# of PSMs')
npsms

In [ ]:
npeptides = [df.drop_duplicates('PeptideSequence').shape[0] for df in dfs]
plt.bar(range(4), npeptides)
plt.xticks([])
plt.title('# of peptides')
npeptides

### Venn diagram of identified peptides

In [ ]:
sets = [set(df.PeptideSequence) for df in dfs]
venn(dict(zip(basenames, sets)))